<a href="https://colab.research.google.com/github/deepali9976/DEL_LAB_1BM22AI041/blob/main/del_lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

multitask learning with
early stopping in a neural network training scenario.
Week6 6Implement a program on Adversarial training, tangent distance, tangent prop
and tangent classifier. [Any three to be implemented].

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


np.random.seed(42)
data = pd.DataFrame({
    'feature1': np.random.rand(1000),
    'feature2': np.random.rand(1000),
    'task1_label': np.random.randint(0, 2, 1000),  # Binary classification
    'task2_label': np.random.randint(0, 10, 1000)  # Multiclass classification
})


X = data[['feature1', 'feature2']].values
y_task1 = data['task1_label'].values
y_task2 = to_categorical(data['task2_label'].values, num_classes=10)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_task1_train, y_task1_val, y_task2_train, y_task2_val = train_test_split(
    X, y_task1, y_task2, test_size=0.2, random_state=42
)

input_layer = Input(shape=(2,))
shared_layer = Dense(64, activation='relu')(input_layer)
shared_layer = Dropout(0.2)(shared_layer)

task1_output = Dense(1, activation='sigmoid', name='task1')(shared_layer)
task2_output = Dense(10, activation='softmax', name='task2')(shared_layer)

model = Model(inputs=input_layer, outputs=[task1_output, task2_output])

model.compile(
    optimizer='adam',
    loss={'task1': 'binary_crossentropy', 'task2': 'categorical_crossentropy'},
    metrics={'task1': 'accuracy', 'task2': 'accuracy'}
)


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


history = model.fit(
    X_train,
    {'task1': y_task1_train, 'task2': y_task2_train},
    validation_data=(X_val, {'task1': y_task1_val, 'task2': y_task2_val}),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 3.0288 - task1_accuracy: 0.4910 - task1_loss: 0.7017 - task2_accuracy: 0.1227 - task2_loss: 2.3271 - val_loss: 2.9974 - val_task1_accuracy: 0.5100 - val_task1_loss: 0.6921 - val_task2_accuracy: 0.1600 - val_task2_loss: 2.3037
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.0049 - task1_accuracy: 0.4758 - task1_loss: 0.6964 - task2_accuracy: 0.1198 - task2_loss: 2.3084 - val_loss: 2.9977 - val_task1_accuracy: 0.5300 - val_task1_loss: 0.6924 - val_task2_accuracy: 0.1450 - val_task2_loss: 2.3061
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.0050 - task1_accuracy: 0.5112 - task1_loss: 0.6940 - task2_accuracy: 0.1102 - task2_loss: 2.3110 - val_loss: 2.9999 - val_task1_accuracy: 0.5000 - val_task1_loss: 0.6933 - val_task2_accuracy: 0.1100 - val_task2_loss: 2.3087
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.9903 - task1_accuracy: 0.5124 - task1_loss: 0.6930 - task2_accuracy: 0.1039 - task2_los

In [3]:
import tensorflow as tf

In [9]:
def generate_adversarial_examples(model, X, y, epsilon=0.1):
    X_adv = X.copy()
    for i in range(len(X)):
        X_sample = tf.convert_to_tensor(X[i:i+1], dtype=tf.float32)
        y_sample = tf.convert_to_tensor(y[i:i+1], dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(X_sample)
            predictions = model(X_sample)
            loss = tf.keras.losses.categorical_crossentropy(y_sample, predictions)
        gradients = tape.gradient(loss, X_sample)
        X_adv[i] += epsilon * np.sign(tf.squeeze(gradients).numpy())  # Fix shape mismatch
    return X_adv

# Train with adversarial examples
X_adv_train = generate_adversarial_examples(model, X_train, y_train)
history = model.fit(
    np.vstack([X_train, X_adv_train]),
    np.vstack([y_train, y_train]),
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=8,
    verbose=1
)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.4935 - loss: 0.6941 - val_accuracy: 0.4500 - val_loss: 0.7145
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4859 - loss: 0.7036 - val_accuracy: 0.4500 - val_loss: 0.7253
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5577 - loss: 0.6854 - val_accuracy: 0.4500 - val_loss: 0.7347
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5571 - loss: 0.6780 - val_accuracy: 0.4500 - val_loss: 0.7444
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5598 - loss: 0.6815 - val_accuracy: 0.4500 - val_loss: 0.7503
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6325 - loss: 0.6534 - val_accuracy: 0.4500 - val_loss: 0.7581
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5839 - loss: 0.6656 - val_accuracy: 0.4500 - val_loss: 0.7608
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5828 - loss: 0.6681 - val_accuracy: 0.4500 - val_los

In [10]:
def tangent_regularization(model, X, tangent_vectors, weight=0.1):
    tangent_loss = 0
    for i in range(len(X)):
        X_sample = tf.convert_to_tensor(X[i:i+1], dtype=tf.float32)
        tangent_vector = tf.convert_to_tensor(tangent_vectors[i:i+1], dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(X_sample)
            predictions = model(X_sample)
        gradients = tape.gradient(predictions, X_sample)
        tangent_loss += tf.reduce_mean(tf.square(tf.reduce_sum(gradients * tangent_vector)))
    return weight * tangent_loss / len(X)

tangent_vectors = np.random.normal(0, 0.01, X_train.shape)

for epoch in range(5):
    with tf.GradientTape() as tape:
        predictions = model(X_train)
        task_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_train, predictions))
        tangent_loss = tangent_regularization(model, X_train, tangent_vectors)
        total_loss = task_loss + tangent_loss
    gradients = tape.gradient(total_loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f"Epoch {epoch+1}, Loss: {total_loss.numpy():.4f}")



Epoch 1, Loss: 0.6626
Epoch 2, Loss: 0.6625
Epoch 3, Loss: 0.6625
Epoch 4, Loss: 0.6624
Epoch 5, Loss: 0.6624


LinAlgError: Incompatible dimensions